In [ ]:
%pip install ortools
%pip install geocoder
%pip install leafmap
%pip install keplergl
%pip install geopandas
%pip install shapely
%pip install polyline shapely

In [5]:
import leafmap.kepler as leafmap
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString
import os
import requests
import polyline
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import random
import datetime
import numpy as np
import pandas as pd
from itertools import permutations
import pandas as pd
import requests
import polyline
import geopandas as gpd
from shapely.geometry import LineString

In [6]:
departure = [33.1521436, -8.6055754]

In [7]:
df = pd.DataFrame({'longitude': [departure[1]] , 'latitude': [departure[0]]})
geometryDep = gpd.GeoSeries.from_xy(df.longitude, df.latitude , crs="EPSG:4326")
m = leafmap.Map(center=departure, zoom=12, height=1000, widescreen=False)
m.add_gdf(geometryDep, "Points", color="red")
m

Map(config={'version': 'v1', 'config': {'mapState': {'latitude': 33.1521436, 'longitude': -8.6055754, 'zoom': …

In [8]:
df = pd.read_csv('DESTINATIONS.csv')
df= df.iloc[[2, 4, 7]]
df = df.reset_index(drop=True)
df

,DESTINATION_NUMBER,DESTINATION_NAME,DESTINATION_LOCATION,LATITUDE,LONGITUDE
0,3,Okay Energy,https://maps.app.goo.gl/t754Yow6ZuL1ab1v6,33.133187,-8.482563
1,6,TotalEnergies ISMAILIYA,https://maps.app.goo.gl/BgtVZb3mvS7W3q5cA,33.899022,-5.541286
2,9,Guiri Afriquia Fuel Station,https://maps.app.goo.gl/ZicpnHz2ye5UPVyM9,33.047938,-8.688203


In [9]:
df.loc[len(df)] = {'LATITUDE': 33.1521436, 'LONGITUDE': -8.6055754, 'DESTINATION_NAME':'DEPARTURE'}
df

,DESTINATION_NUMBER,DESTINATION_NAME,DESTINATION_LOCATION,LATITUDE,LONGITUDE
0,3.0,Okay Energy,https://maps.app.goo.gl/t754Yow6ZuL1ab1v6,33.133187,-8.482563
1,6.0,TotalEnergies ISMAILIYA,https://maps.app.goo.gl/BgtVZb3mvS7W3q5cA,33.899022,-5.541286
2,9.0,Guiri Afriquia Fuel Station,https://maps.app.goo.gl/ZicpnHz2ye5UPVyM9,33.047938,-8.688203
3,NaN,DEPARTURE,NaN,33.152144,-8.605575


In [10]:
# Merge df with itself to create all possible pairs
merged = pd.merge(df, df, how='cross')

# Exclude rows where DEPARTURE is the same as DESTINATION
merged = merged[merged['DESTINATION_NAME_x'] != merged['DESTINATION_NAME_y']]

# Select relevant columns
result = merged[['DESTINATION_NAME_x', 'LATITUDE_x', 'LONGITUDE_x', 'DESTINATION_NAME_y', 'LATITUDE_y', 'LONGITUDE_y']]

# Rename columns
result.columns = ['DEPARTURE', 'LATITUDE_departure', 'LONGITUDE_departure', 'DESTINATION', 'LATITUDE_destination', 'LONGITUDE_destination']

result

,DEPARTURE,LATITUDE_departure,LONGITUDE_departure,DESTINATION,LATITUDE_destination,LONGITUDE_destination
1,Okay Energy,33.133187,-8.482563,TotalEnergies ISMAILIYA,33.899022,-5.541286
2,Okay Energy,33.133187,-8.482563,Guiri Afriquia Fuel Station,33.047938,-8.688203
3,Okay Energy,33.133187,-8.482563,DEPARTURE,33.152144,-8.605575
4,TotalEnergies ISMAILIYA,33.899022,-5.541286,Okay Energy,33.133187,-8.482563
6,TotalEnergies ISMAILIYA,33.899022,-5.541286,Guiri Afriquia Fuel Station,33.047938,-8.688203
7,TotalEnergies ISMAILIYA,33.899022,-5.541286,DEPARTURE,33.152144,-8.605575
8,Guiri Afriquia Fuel Station,33.047938,-8.688203,Okay Energy,33.133187,-8.482563
9,Guiri Afriquia Fuel Station,33.047938,-8.688203,TotalEnergies ISMAILIYA,33.899022,-5.541286
11,Guiri Afriquia Fuel Station,33.047938,-8.688203,DEPARTURE,33.152144,-8.605575
12,DEPARTURE,33.152144,-8.605575,Okay Energy,33.133187,-8.482563


In [11]:
def get_distance_duration(departure, destination):
    # Validate input data
    if 'LONGITUDE_departure' not in departure or 'LATITUDE_departure' not in departure:
        raise ValueError("Departure DataFrame must contain 'LONGITUDE_departure' and 'LATITUDE_departure' columns.")
    if 'LONGITUDE_destination' not in destination or 'LATITUDE_destination' not in destination:
        raise ValueError("Destination DataFrame must contain 'LONGITUDE_destination' and 'LATITUDE_destination' columns.")

    # Construct URL for OSRM API request
    url = f"https://router.project-osrm.org/route/v1/driving/{departure['LONGITUDE_departure']},{departure['LATITUDE_departure']};{destination['LONGITUDE_destination']},{destination['LATITUDE_destination']}?overview=full"

    # Send GET request to OSRM API
    r = requests.get(url)
    
    # Check if request was successful
    if r.status_code != 200:
        raise ValueError(f"Error fetching data from OSRM API: {r.text}")

    # Parse JSON response
    data = r.json()

    # Extract distance and duration from response
    distance = data['routes'][0]['distance']
    duration = data['routes'][0]['duration']
    encoded_polyline = data['routes'][0]['geometry']
    return distance, duration , encoded_polyline

In [12]:
# Iterate through each row of the DataFrame
for index, row in result.iterrows():
    # Create DataFrames for departure and destination
    departures = {'LATITUDE_departure': row['LATITUDE_departure'], 'LONGITUDE_departure': row['LONGITUDE_departure']}
    destination = {'LATITUDE_destination': row['LATITUDE_destination'], 'LONGITUDE_destination': row['LONGITUDE_destination']}
    
    # Calculate distance, duration, and decoded polyline
    distance, duration, encoded_polyline = get_distance_duration(departures, destination)
    
    # Assign distance, duration, and decoded polyline to new columns in the DataFrame
    result.at[index, 'DISTANCE'] = distance
    result.at[index, 'DURATION'] = duration
    result.at[index, 'ENCODED_POLYLINE'] = encoded_polyline

result
result.to_csv('Destinations_weight_tsp.csv')
#result = pd.read_csv('Destinations_weight_tsp.csv')

In [13]:
result

,DEPARTURE,LATITUDE_departure,LONGITUDE_departure,DESTINATION,LATITUDE_destination,LONGITUDE_destination,DISTANCE,DURATION,ENCODED_POLYLINE
1,Okay Energy,33.133187,-8.482563,TotalEnergies ISMAILIYA,33.899022,-5.541286,345755.3,14010.0,ijfiEvuwr@i@h@]b@ODmE?ONk@?yBS}Ce@IAcDq@mCk@y@...
2,Okay Energy,33.133187,-8.482563,Guiri Afriquia Fuel Station,33.047938,-8.688203,30419.1,2271.7,ijfiEvuwr@i@h@]b@ODmE?ONlABvA?zA?hDBjABJ@dBN|D...
3,Okay Energy,33.133187,-8.482563,DEPARTURE,33.152144,-8.605575,19782.8,1411.1,ijfiEvuwr@i@h@]b@ODmE?ONk@?yBS}Ce@IAcDq@mCk@y@...
4,TotalEnergies ISMAILIYA,33.899022,-5.541286,Okay Energy,33.133187,-8.482563,347318.5,14105.9,u{{mE`hy`@QzCbGb@JUPe@Pm@TaAx@JTBFf@T|AXrBb@bD...
6,TotalEnergies ISMAILIYA,33.899022,-5.541286,Guiri Afriquia Fuel Station,33.047938,-8.688203,368420.1,15346.9,u{{mE`hy`@QzCbGb@JUPe@Pm@TaAx@JTBFf@T|AXrBb@bD...
7,TotalEnergies ISMAILIYA,33.899022,-5.541286,DEPARTURE,33.152144,-8.605575,358891.1,14559.1,u{{mE`hy`@QzCbGb@JUPe@Pm@TaAx@JTBFf@T|AXrBb@bD...
8,Guiri Afriquia Fuel Station,33.047938,-8.688203,Okay Energy,33.133187,-8.482563,30463.7,2278.6,otuhEl{_t@{AUiBYcB[{@O_Bi@w@e@k@i@k@o@eAgAgAyA...
9,Guiri Afriquia Fuel Station,33.047938,-8.688203,TotalEnergies ISMAILIYA,33.899022,-5.541286,367675.7,15309.4,otuhEl{_t@{AUiBYcB[{@O_Bi@w@e@k@i@k@o@eAgAgAyA...
11,Guiri Afriquia Fuel Station,33.047938,-8.688203,DEPARTURE,33.152144,-8.605575,14829.3,1038.2,otuhEl{_t@{AUiBYcB[{@O_Bi@w@e@k@i@k@o@eAgAgAyA...
12,DEPARTURE,33.152144,-8.605575,Okay Energy,33.133187,-8.482563,20706.3,1474.0,q_jiEfvos@bDn@dJhBzBd@pDt@|@TpB\tDt@jCh@bB\vDz...


In [14]:
# Filter the DataFrame to exclude rows with DESTINATION_NAME 'DEPARTURE'
filtered_df = df[df['DESTINATION_NAME'] != 'DEPARTURE']

# Calculate the length of the filtered destinations
num_destinations = len(filtered_df)

# Generate random percentages that sum up to 100
order_quantity_percentages = [random.randint(1, 100) for _ in range(num_destinations)]
total_percentage = sum(order_quantity_percentages)
order_quantity_percentages = [percentage * 100 / total_percentage for percentage in order_quantity_percentages]

# Create the Client_for_delivery DataFrame
Client_for_delivery = pd.DataFrame({
    'Order_quantity_percentage': order_quantity_percentages,
    'DESTINATION_NAME': filtered_df['DESTINATION_NAME']
})

Client_for_delivery

,Order_quantity_percentage,DESTINATION_NAME
0,32.748538,Okay Energy
1,50.292398,TotalEnergies ISMAILIYA
2,16.959064,Guiri Afriquia Fuel Station


In [15]:
# Function to calculate fuel consumption
empty_truck_consumption = 27  # in L/100KM
full_truck_consumption = 40   # in L/100KM 

def display_time_from_seconds(message,seconds):
    # Convert seconds to a timedelta object
    time_delta = datetime.timedelta(seconds=seconds)
    
    # Convert the timedelta object to hours, minutes, and seconds
    hours = time_delta.seconds // 3600
    minutes = (time_delta.seconds // 60) % 60
    seconds = time_delta.seconds % 60
    
    # Format and display the time
    print(message,": {:02d}:{:02d}:{:02d}".format(hours, minutes, seconds))

def calculate_consumption1(order_percentage, dep, dest, actual_load_percentage):
    if dest == 'DEPARTURE':
        return empty_truck_consumption
    if dep == 'DEPARTURE':
        return full_truck_consumption
    
    # Calculate adjusted consumption based on load percentage
    adjusted_load_percentage = actual_load_percentage - order_percentage
    adjusted_consumption = empty_truck_consumption + (full_truck_consumption - empty_truck_consumption) * (actual_load_percentage / 100)
    return adjusted_consumption

def normalize(value, max_value):
    return value / max_value

In [16]:
destination_permutations = permutations(Client_for_delivery['DESTINATION_NAME'])

best_path = None
best_consumption = float('inf')
best_score = float('inf')
best_duration = float('inf')
best_distance= float('inf')
max_consumption = 0
max_duration = 0

for path in destination_permutations:
    # Reset variables for each permutation
    current_location = 'DEPARTURE'
    visited_destinations = ['DEPARTURE']
    total_consumption = 0
    total_duration=0
    total_score=0
    actual_truck_load_percentage = 100
    for destination in path:
        duration = result.loc[(result['DEPARTURE'] == current_location) & (result['DESTINATION'] == destination), 'DURATION'].iloc[0]
        distance = result.loc[(result['DEPARTURE'] == current_location) & (result['DESTINATION'] == destination), 'DISTANCE'].iloc[0]
        order_percentage = Client_for_delivery.loc[Client_for_delivery['DESTINATION_NAME'] == destination, 'Order_quantity_percentage'].iloc[0]
        consumption = calculate_consumption1(order_percentage, current_location, destination, actual_truck_load_percentage)
        total_consumption += consumption * (distance / 1000) /100 # Convert distance from meters to kilometers
        total_duration += duration
        current_location = destination
        visited_destinations.append(destination)
        actual_truck_load_percentage -= order_percentage
        
    # Return to the departure location
    distance_to_start = result.loc[(result['DEPARTURE'] == current_location) & (result['DESTINATION'] == 'DEPARTURE'), 'DISTANCE'].iloc[0]
    duration_to_start = result.loc[(result['DEPARTURE'] == current_location) & (result['DESTINATION'] == 'DEPARTURE'), 'DURATION'].iloc[0]
    consumption_to_start = empty_truck_consumption * (distance_to_start / 1000) /100
    total_consumption += consumption_to_start
    total_duration += duration_to_start
    # Update max_consumption and max_duration if needed
    if total_consumption > max_consumption:
        max_consumption = total_consumption
    if total_duration > max_duration:
        max_duration = total_duration

In [17]:
# Generate all possible permutations of destinations
destination_permutations = permutations(Client_for_delivery['DESTINATION_NAME'])
# Iterate through each permutation
i = 1
for path in destination_permutations:
    # Reset variables for each permutation
    current_location = 'DEPARTURE'
    visited_destinations = ['DEPARTURE']
    total_consumption = 0
    total_duration=0
    total_score=0
    total_distance=0
    actual_truck_load_percentage = 100
    # Calculate total consumption for the permutation
    for destination in path:
        duration = result.loc[(result['DEPARTURE'] == current_location) & (result['DESTINATION'] == destination), 'DURATION'].iloc[0]
        distance = result.loc[(result['DEPARTURE'] == current_location) & (result['DESTINATION'] == destination), 'DISTANCE'].iloc[0]
        order_percentage = Client_for_delivery.loc[Client_for_delivery['DESTINATION_NAME'] == destination, 'Order_quantity_percentage'].iloc[0]
        consumption = calculate_consumption1(order_percentage, current_location, destination, actual_truck_load_percentage)
        total_duration+=duration
        total_consumption += consumption * (distance / 1000) /100 # Convert distance from meters to kilometers
        current_location = destination
        total_distance +=distance
        visited_destinations.append(destination)
        actual_truck_load_percentage -= order_percentage
    
    # Return to the departure location
    distance_to_start = result.loc[(result['DEPARTURE'] == current_location) & (result['DESTINATION'] == 'DEPARTURE'), 'DISTANCE'].iloc[0]
    duration_to_start = result.loc[(result['DEPARTURE'] == current_location) & (result['DESTINATION'] == 'DEPARTURE'), 'DURATION'].iloc[0]
    consumption_to_start = empty_truck_consumption * (distance_to_start / 1000) /100
    total_consumption += consumption_to_start
    total_duration += duration_to_start
    total_distance +=distance_to_start
    # Normalize the values
    normalized_consumption = normalize(total_consumption, max_consumption)
    normalized_duration = normalize(total_duration, max_duration)
    # Calculate the total score
    total_score = normalized_consumption + normalized_duration
    visited_destinations.append('DEPARTURE')
    
    print("Path : ", i)
    print("distance:",total_distance/1000)
    print("consumption:",total_consumption)
    display_time_from_seconds("Duration",total_duration)
    print("----------------")
    
    i+=1
    # Update the best path and minimum consumption if this permutation is better
    if total_score < best_score:
        best_consumption = total_consumption
        best_path = visited_destinations
        best_score = total_score
        best_duration = total_duration
        best_distance = total_distance

#print the best path and minimum consumption
print("Best Path:", best_path)
print("Best Distance:", best_distance/1000)
print("best Consumption:", best_consumption)
display_time_from_seconds("best Duration",best_duration)
print("Best Score:", best_score)

Path :  1
distance: 749.711
consumption: 243.4645814678362
Duration : 08:51:09
----------------
Path :  2
distance: 777.6922
consumption: 239.3668409005848
Duration : 09:20:14
----------------
Path :  3
distance: 750.4852000000001
consumption: 272.2747071637427
Duration : 08:52:00
----------------
Path :  4
distance: 776.5849
consumption: 281.3114999005848
Duration : 09:19:01
----------------
Path :  5
distance: 749.5766
consumption: 230.1605033508772
Duration : 08:51:05
----------------
Path :  6
distance: 749.2435
consumption: 258.6545894269006
Duration : 08:50:44
----------------
Best Path: ['DEPARTURE', 'Guiri Afriquia Fuel Station', 'Okay Energy', 'TotalEnergies ISMAILIYA', 'DEPARTURE']
Best Distance: 749.5766
best Consumption: 230.1605033508772
best Duration : 08:51:05
Best Score: 1.7661498680814236


In [18]:
import leafmap
import polyline
from shapely.geometry import LineString
import geopandas as gpd

# Define a list of colors
colors = ["red", "green", "blue", "yellow", "orange", "purple"]  # Add more colors if needed

df_copy = df.copy()

# Define a default center for the map
default_center = [33.1521436, -8.6055754]  # Set appropriate latitude and longitude values
# Initialize the map with the default center
m = leafmap.Map(center=default_center, zoom=12, height=600, widescreen=False)

# Iterate through each pair of points in the best_path
for i in range(len(best_path)-1):
    try:
        x = best_path[i]
        y = best_path[i+1]
        encoded_polyline = result.loc[(result['DEPARTURE'] == x) & (result['DESTINATION'] == y), 'ENCODED_POLYLINE'].iloc[0]
        decoded_polyline = polyline.decode(encoded_polyline, 5)
        decoded_polyline = [t[::-1] for t in decoded_polyline]
        route = gpd.GeoDataFrame(geometry=[LineString(decoded_polyline)], crs='EPSG:4326')
        
        # Use a color from the predefined list, cycling through it
        color = colors[i % len(colors)]
        
        m.add_gdf(route, "route{}".format(i+1), style_callback=lambda feat, color=color: {"color": color})
    except Exception as e:
        print(f"Error processing route {i+1}: {e}")

# Add points to the map
geometry = gpd.GeoSeries.from_xy(df_copy['LONGITUDE'], df_copy['LATITUDE'], crs="EPSG:4326")
m.add_gdf(geometry.to_frame(), "Points")

m


Map(center=[33.1521436, -8.6055754], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…